In [1]:
import os
import socket
from datetime import datetime, timezone

import dask
import dask_geopandas as dgpd
from dask.distributed import Client




from metrics_calculation import (
    building_and_intersection_metrics,
    building_distance_metrics,
    compute_m6_m7,
    metrics_roads_intersections,
    metrics_k_blocks,
)

from standardize_metrics import *




In [2]:
import coiled
from dask.distributed import Client

YOUR_NAME = "sara"
AWS_PROFILE = 'cities'

import os
os.environ['AWS_PROFILE'] = AWS_PROFILE

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f"ils-{YOUR_NAME}",
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=10,
    package_sync_ignore=["pyspark", "pypandoc"],
    worker_options={"nthreads": 1},
)

CLIENT = cluster.get_client()
print("Coiled dashboard:", CLIENT.dashboard_link)

def check_creds():
    import os
    return {
        "AWS_ACCESS_KEY_ID": bool(os.getenv("AWS_ACCESS_KEY_ID")),
        "AWS_SECRET_ACCESS_KEY": bool(os.getenv("AWS_SECRET_ACCESS_KEY")),
        "AWS_SESSION_TOKEN": bool(os.getenv("AWS_SESSION_TOKEN")),
        "AWS_DEFAULT_REGION": os.getenv("AWS_DEFAULT_REGION"),
    }

print(CLIENT.run(check_creds))


[2026-01-28 16:23:34,214][INFO    ][coiled] Fetching latest package priorities...
[2026-01-28 16:23:34,215][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2026-01-28 16:23:34,716][INFO    ][coiled.package_sync] Scanning 446 conda packages...
[2026-01-28 16:23:34,719][INFO    ][coiled.package_sync] Scanning 261 python packages...
[2026-01-28 16:23:35,114][INFO    ][coiled.software_utils] No username or password found for https://conda.anaconda.org/conda-forge
[2026-01-28 16:23:35,637][INFO    ][coiled] Running pip check...
[2026-01-28 16:23:36,102][INFO    ][coiled] Validating environment...
[2026-01-28 16:23:40,374][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2026-01-28 16:23:40,471][INFO    ][coiled] Creating wheel for /opt/spark-2.2.0/python...
[2026-01-28 16:23:40,628][WARNING ][coiled.package_sync] Package - debugpy, debugpy~=1.8.16 has no install candidate for Python 3.12 l

Coiled dashboard: https://cluster-hcmbo.dask.host/Kqoc9aZo_0lY8SuB/status
{'tls://10.0.195.12:43701': {'AWS_ACCESS_KEY_ID': False, 'AWS_SECRET_ACCESS_KEY': False, 'AWS_SESSION_TOKEN': False, 'AWS_DEFAULT_REGION': None}, 'tls://10.0.195.32:37535': {'AWS_ACCESS_KEY_ID': False, 'AWS_SECRET_ACCESS_KEY': False, 'AWS_SESSION_TOKEN': False, 'AWS_DEFAULT_REGION': None}, 'tls://10.0.198.228:43185': {'AWS_ACCESS_KEY_ID': False, 'AWS_SECRET_ACCESS_KEY': False, 'AWS_SESSION_TOKEN': False, 'AWS_DEFAULT_REGION': None}, 'tls://10.0.201.176:39793': {'AWS_ACCESS_KEY_ID': False, 'AWS_SECRET_ACCESS_KEY': False, 'AWS_SESSION_TOKEN': False, 'AWS_DEFAULT_REGION': None}, 'tls://10.0.202.18:46495': {'AWS_ACCESS_KEY_ID': False, 'AWS_SECRET_ACCESS_KEY': False, 'AWS_SESSION_TOKEN': False, 'AWS_DEFAULT_REGION': None}, 'tls://10.0.203.254:33057': {'AWS_ACCESS_KEY_ID': False, 'AWS_SECRET_ACCESS_KEY': False, 'AWS_SESSION_TOKEN': False, 'AWS_DEFAULT_REGION': None}, 'tls://10.0.206.138:33547': {'AWS_ACCESS_KEY_ID': Fa

/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:1590: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.4.4  | 4.4.5     | 4.4.5   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: 

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
ROADS_PATH = f'{INPUT_PATH}/roads'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
from datetime import datetime
RUN_ID = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
LOGS_S3_DIR = f"{OUTPUT_PATH}/logs/metric_calculation/{RUN_ID}"
SUMMARY_S3_PATH = f"{LOGS_S3_DIR}/summary.csv"

def write_json_to_s3(obj, s3_path: str):
    import json, s3fs
    fs = s3fs.S3FileSystem(anon=False)
    with fs.open(s3_path, "w") as f:
        json.dump(obj, f, indent=2, default=str)


/var/folders/nn/3mdkp6sx1n3d955f3wqgdbb00000gn/T/ipykernel_75934/3399571734.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  RUN_ID = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")


In [5]:
from standardize_metrics import *

def apply_metric_standardization(df):
    """
    Adds m{i}_std from m{i}_raw using the per-metric functions in standardize_metrics.py.
    Expects keys like 'metric_1', 'metric_2', ...
    """
    import re

    for key, std_func in standardization_functions.items():
        m = re.match(r"metric_(\d+)$", str(key))
        if not m:
            continue

        i = int(m.group(1))
        raw_col = f"m{i}_raw"
        std_col = f"m{i}_std"

        if raw_col not in df.columns:
            continue

        df[std_col] = df[raw_col].map_partitions(
            std_func,
            meta=(std_col, "float64")
        )

    return df



In [6]:
import os, uuid
import dask_geopandas as dgpd
import geopandas as gpd


def safe_merge_on_index(left, right, right_prefix=None):
    """
    Merge right into left on index without suffix collisions.
    - Drops geometry from right
    - If right_prefix is provided, prefixes ALL right columns before merge
    - Drops any overlaps to avoid MergeError from duplicate suffixes
    """
    if right is None:
        return left

    right2 = right.drop(columns="geometry", errors="ignore")

    if right_prefix:
        right2 = right2.rename(columns={c: f"{right_prefix}{c}" for c in right2.columns})

    overlap = [c for c in right2.columns if c in left.columns]
    if overlap:
        right2 = right2.drop(columns=overlap, errors="ignore")

    return left.merge(right2, left_index=True, right_index=True, how="left")


def write_parquet_to_s3_atomic(gdf, out_s3_uri: str, fs, city_name: str):
    """
    Robust single-file S3 write:
    local -> put(tmp) -> mv(tmp->final)
    Prevents 0-byte final objects.
    """
    local_tmp = f"/tmp/{city_name}_{uuid.uuid4().hex}.geoparquet"
    out_tmp = out_s3_uri + ".tmp"

    # remove 0-byte leftover at final if any
    try:
        if fs.exists(out_s3_uri) and fs.info(out_s3_uri).get("Size", 0) == 0:
            fs.rm(out_s3_uri)
    except Exception:
        pass

    # write local first
    gdf.to_parquet(local_tmp, engine="pyarrow", index=False)

    # upload then swap into place
    fs.put(local_tmp, out_tmp)
    fs.mv(out_tmp, out_s3_uri)

    try:
        os.remove(local_tmp)
    except Exception:
        pass


def consolidate_city_to_all(city_name: str, YOUR_NAME: str):
    """
    PASS 2:
    Read the per-metric-group outputs, merge safely (k_* prefixed),
    add *_std (per-city), and write ONE consolidated file.
    """
    base = f"{OUTPUT_PATH_RASTER}/{city_name}/{city_name}"

    p_12        = f"{base}_block_metrics_1_2_{YOUR_NAME}"
    p_345101112 = f"{base}_block_metrics_3_4_5_10_11_12_{YOUR_NAME}"
    p_67        = f"{base}_block_metrics_6_7_{YOUR_NAME}"
    p_89        = f"{base}_block_metrics_8_9_{YOUR_NAME}"
    p_k         = f"{base}_block_metrics_k_{YOUR_NAME}.geoparquet"

    out = f"{base}_block_metrics_ALL_{YOUR_NAME}.geoparquet"

    # skip if exists and non-empty
    try:
        if fs.exists(out) and fs.info(out).get("Size", 0) > 0:
            return out
    except Exception:
        pass

    # geometry master
    g_base = dgpd.read_parquet(p_345101112)

    g_12 = dgpd.read_parquet(p_12)
    g_67 = dgpd.read_parquet(p_67)
    g_89 = dgpd.read_parquet(p_89)
    g_k  = dgpd.read_parquet(p_k)

    df = g_base
    df = safe_merge_on_index(df, g_12)
    df = safe_merge_on_index(df, g_67)
    df = safe_merge_on_index(df, g_89)

    # prefix ALL k-block cols to avoid collisions (optimal_point, max_radius, etc.)
    df = safe_merge_on_index(df, g_k, right_prefix="k_")

    # per-city standardization (NOT global)
    df = apply_metric_standardization(df)

    pdf = df.compute()
    gdf = gpd.GeoDataFrame(pdf, geometry="geometry", crs=g_base.crs)

    write_parquet_to_s3_atomic(gdf, out, fs, city_name=city_name)
    return out


In [7]:
def _s3_exists(s3_uri: str) -> bool:
    return fs.exists(s3_uri)


def run_metrics_for_city(city_name: str):
    """
    PASS 1 (metrics-only): run metric group writers.
    Do NOT merge/std/write _ALL_ here.
    """
    import traceback
    from datetime import datetime, timezone

    start = datetime.now(timezone.utc)
    host = socket.gethostname()

    rec = {
        "utc_start": start.isoformat(),
        "utc_end": None,
        "seconds": None,
        "city": city_name,
        "host": host,
        "status": "started",

        "p12": None,
        "p345101112": None,
        "p67": None,
        "p89": None,
        "pk": None,

        "m12_ok": False,
        "m345101112_ok": False,
        "m67_ok": False,
        "m89_ok": False,
        "mk_ok": False,

        "m12_error": None,
        "m345101112_error": None,
        "m67_error": None,
        "m89_error": None,
        "mk_error": None,
    }

    city_log_path = f"{LOGS_S3_DIR}/cities/{city_name}.json"

    try:
        p12 = p345101112 = p67 = p89 = pk = None

        try:
            p12 = building_distance_metrics(city_name, YOUR_NAME)
            rec["m12_ok"] = True
        except Exception as e:
            rec["m12_error"] = f"{type(e).__name__}: {e}\n{traceback.format_exc()}"

        try:
            p345101112 = building_and_intersection_metrics(city_name, YOUR_NAME)
            rec["m345101112_ok"] = True
        except Exception as e:
            rec["m345101112_error"] = f"{type(e).__name__}: {e}\n{traceback.format_exc()}"

        try:
            p67 = compute_m6_m7(city_name, YOUR_NAME)
            rec["m67_ok"] = True
        except Exception as e:
            rec["m67_error"] = f"{type(e).__name__}: {e}\n{traceback.format_exc()}"

        try:
            p89 = metrics_roads_intersections(city_name, YOUR_NAME)
            rec["m89_ok"] = True
        except Exception as e:
            rec["m89_error"] = f"{type(e).__name__}: {e}\n{traceback.format_exc()}"

        try:
            pk = metrics_k_blocks(city_name, YOUR_NAME)
            rec["mk_ok"] = True
        except Exception as e:
            rec["mk_error"] = f"{type(e).__name__}: {e}\n{traceback.format_exc()}"

        # materialize returned paths
        p12, p345101112, p67, p89, pk = dask.compute(p12, p345101112, p67, p89, pk)

        rec["p12"] = p12
        rec["p345101112"] = p345101112
        rec["p67"] = p67
        rec["p89"] = p89
        rec["pk"] = pk

        end = datetime.now(timezone.utc)
        rec["utc_end"] = end.isoformat()
        rec["seconds"] = (end - start).total_seconds()

        rec["status"] = (
            "ok_with_warnings"
            if any(rec.get(k) for k in ["m12_error", "m345101112_error", "m67_error", "m89_error", "mk_error"])
            else "ok"
        )
        return rec

    finally:
        try:
            write_json_to_s3(rec, city_log_path)
        except Exception:
            pass


def run_consolidation_for_city(city_name: str):
    """
    PASS 2: merge + std + write _ALL_ atomically
    """
    import traceback
    from datetime import datetime, timezone

    start = datetime.now(timezone.utc)
    host = socket.gethostname()

    out = f"{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_block_metrics_ALL_{YOUR_NAME}.geoparquet"
    city_log_path = f"{LOGS_S3_DIR}/cities/{city_name}_consolidate.json"

    rec = {
        "utc_start": start.isoformat(),
        "utc_end": None,
        "seconds": None,
        "city": city_name,
        "host": host,
        "status": "started",
        "output": out,
        "error": None,
    }

    try:
        # skip if exists and non-empty
        try:
            if fs.exists(out) and fs.info(out).get("Size", 0) > 0:
                rec["status"] = "skipped_exists"
                end = datetime.now(timezone.utc)
                rec["utc_end"] = end.isoformat()
                rec["seconds"] = (end - start).total_seconds()
                return rec
        except Exception:
            pass

        consolidate_city_to_all(city_name, YOUR_NAME)

        end = datetime.now(timezone.utc)
        rec["utc_end"] = end.isoformat()
        rec["seconds"] = (end - start).total_seconds()
        rec["status"] = "ok"
        return rec

    except Exception as e:
        rec["status"] = "failed"
        rec["error"] = f"{type(e).__name__}: {e}\n{traceback.format_exc()}"
        return rec

    finally:
        try:
            write_json_to_s3(rec, city_log_path)
        except Exception:
            pass


In [8]:
import s3fs

fs = s3fs.S3FileSystem(anon=False)

IGNORE_CITIES = [
    'Ngo__Nigeria',
    'Bugama__Nigeria',
    'Mubi__Nigeria',
    'San_Pedro_de_Macoris__Dominican_Republic',
]


# Each city folder is under BLOCS_PATH/<city>/...
block_city_dirs = fs.ls(BLOCKS_PATH)
all_cities = sorted([p.split('/')[-1] for p in block_city_dirs])

# remove empty / sentinel items if any
all_cities = [c for c in all_cities if c and c not in ['.DS_Store']]

# ignore the 4 cities you want to skip
cities = [c for c in all_cities if c not in set(IGNORE_CITIES)]

print('roads city dirs:', len(all_cities))
print('cities to run:', len(cities))



# -------------------------
# PASS 1: METRICS
# -------------------------
futures = CLIENT.map(run_metrics_for_city, cities)
results_metrics = CLIENT.gather(futures)

metrics_df = pd.DataFrame(results_metrics)

def write_csv_to_s3(df, s3_path):
    import s3fs
    fs = s3fs.S3FileSystem(anon=False)
    with fs.open(s3_path, "w") as f:
        df.to_csv(f, index=False)

CLIENT.submit(write_csv_to_s3, metrics_df, f"{LOGS_S3_DIR}/summary_metrics.csv").result()
print("Wrote metrics summary:", f"{LOGS_S3_DIR}/summary_metrics.csv")


# -------------------------
# PASS 2: CONSOLIDATION
# -------------------------
futures2 = CLIENT.map(run_consolidation_for_city, cities)
results_cons = CLIENT.gather(futures2)

cons_df = pd.DataFrame(results_cons)
CLIENT.submit(write_csv_to_s3, cons_df, f"{LOGS_S3_DIR}/summary_consolidation.csv").result()
print("Wrote consolidation summary:", f"{LOGS_S3_DIR}/summary_consolidation.csv")

results_cons


roads city dirs: 1234
cities to run: 1234
Wrote metrics summary: s3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/logs/metric_calculation/20260128T212541Z/summary_metrics.csv
Wrote consolidation summary: s3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/logs/metric_calculation/20260128T212541Z/summary_consolidation.csv


[{'utc_start': '2026-01-29T00:50:50.305911+00:00',
  'utc_end': '2026-01-29T00:50:57.465015+00:00',
  'seconds': 7.159104,
  'city': 'Aba__Nigeria',
  'host': 'ip-10-0-203-254',
  'status': 'ok',
  'output': 's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Aba__Nigeria/Aba__Nigeria_block_metrics_ALL_sara.geoparquet',
  'error': None},
 {'utc_start': '2026-01-29T00:50:50.308356+00:00',
  'utc_end': '2026-01-29T00:50:55.586236+00:00',
  'seconds': 5.27788,
  'city': 'Abacher__Chad',
  'host': 'ip-10-0-201-176',
  'status': 'ok',
  'output': 's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/raster/Abacher__Chad/Abacher__Chad_block_metrics_ALL_sara.geoparquet',
  'error': None},
 {'utc_start': '2026-01-29T00:50:50.311189+00:00',
  'utc_end': '2026-01-29T00:50:55.675781+00:00',
  'seconds': 5.364592,
  'city': 'Abakaliki__Nigeria',
  'host': 'ip-10-0-202-18',
  'status': 'ok',
  'output': 's3://wri-cities-sandbox/identifyingLandSubdivisions/data/output/